In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.ensemble import IsolationForest
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from math import ceil
from tqdm import tqdm
import warnings
from matplotlib.font_manager import FontProperties
from collections import defaultdict
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 300)

from IPython.core.display import display, HTML # расширяем блокнот
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None) # выводим все колонки

from IPython.core.interactiveshell import InteractiveShell # выводим все результаты из ячейки
InteractiveShell.ast_node_interactivity = "all"

In [35]:
df = pd.read_csv('data/data1.csv')
del df['Unnamed: 0']
del df['DateTime']
df = df.fillna(0)

In [3]:
def cumsum_os_ha(data: pd.DataFrame):
    """Function for calculating the cumulative sum for columns ph and pa, depending on the values of H AND A, respectively, for each S (p1 and p2)
    p5 and p6 - the same regardless of S
    :param df: Original DataFrame
    :return: cumsum by columns
    """
    data['p1'] = data.groupby(['S','H'])['ph'].transform(lambda x: x.cumsum().shift())
    data['p2'] = data.groupby(['S','A'])['pa'].transform(lambda x: x.cumsum().shift())
    data['p5'] = data.groupby(['H'])['ph'].transform(lambda x: x.cumsum().shift())
    data['p6'] = data.groupby(['A'])['pa'].transform(lambda x: x.cumsum().shift())
    return data

In [5]:
def cumsum_o_ha(data: pd.DataFrame):
    """Function for calculating the cumulative sum for columns ph and pa, depending on the values of H OR A, respectively, for each S 
    :param df: Original DataFrame
    :return: cumsum by columns
    """
    data['id'] = range(1, len(data) + 1)
    data1 = data[['S', 'H', 'ph', 'id']]
    data2 = data[['S', 'A', 'pa', 'id']]
    data2.rename(columns={'A':'H', 'pa': 'ph'}, inplace=True)
    data3 = pd.concat([data1, data2])
    data3.sort_values('id', inplace=True)
    data3['p3'] = data3.groupby(['S','H'])['ph'].transform(lambda x: x.cumsum().shift())
    data = data.merge(data3, left_on=('S', 'H', 'id', 'ph'), right_on=('S', 'H', 'id', 'ph'))
    data3.rename(columns={'H':'A', 'ph': 'pa'}, inplace=True)
    data = data.merge(data3, left_on=('S', 'A', 'id', 'pa'), right_on=('S', 'A', 'id', 'pa'))
    data.rename(columns={'p3_x': 'p3', 'p3_y': 'p4'}, inplace=True)
    return data

In [8]:
def cumsum_o(data: pd.DataFrame):
    data['id'] = range(1, len(data) + 1)
    data1 = data[['H', 'ph', 'id']]
    data2 = data[['A', 'pa', 'id']]
    data2.rename(columns={'A':'H', 'pa': 'ph'}, inplace=True)
    data3 = pd.concat([data1, data2])
    data3.sort_values('id', inplace=True)
    data3['p7'] = data3.groupby(['H'])['ph'].transform(lambda x: x.cumsum().shift())
    data = data.merge(data3, left_on=('H', 'id', 'ph'), right_on=('H', 'id', 'ph'))
    data3.rename(columns={'H':'A', 'ph': 'pa'}, inplace=True)
    data = data.merge(data3, left_on=('A', 'id', 'pa'), right_on=('A', 'id', 'pa'))
    data.rename(columns={'p7_x': 'p7', 'p7_y': 'p8'}, inplace=True)
    return data

In [10]:
def create_p_1234(df: pd.DataFrame):
    """Function for new columns
    :param df: Original DataFrame
    :return: new df
    """
    df['p1.1'] = df.ph + df.p1
    df['p2.1'] = df.pa + df.p2
    df['p3.1'] = df.ph + df.p3
    df['p4.1'] = df.pa + df.p4
    return df

In [12]:
def function_T(data: pd.DataFrame):
    df = data[['S', 'H', 'A']]
    df['TH'], df['TA'] = 0, 0
    my_dict = {}
    for i in pd.unique(df[['H', 'A']].values.ravel('K')):
        my_dict[i] = 0

    for S in df.S.unique():
        for ind in df[df.S == S][['H', 'A']].index:
            for num in my_dict:
                if df.iloc[ind].H == num:        
                    my_dict[df.iloc[ind].H] += 1
                    df.iloc[ind].TH = my_dict[df.iloc[ind].H]
                elif df.iloc[ind].A == num: 
                    my_dict[df.iloc[ind].A] += 1
                    df.iloc[ind].TA = my_dict[df.iloc[ind].A]
        for i in pd.unique(df[['H', 'A']].values.ravel('K')):
            my_dict[i] = 0
    data = data.drop(columns=['S', 'H', 'A'])
    return pd.concat([df, data], axis=1)

In [ ]:
    data2['ta'], data2['th'] = 1, 1
    for S in data2.S.unique():
        for A in data2.A.unique():
            data2.loc[(data2.S == S) & (data2.A == A), 'ta'] = data2.loc[(data2.S == S) & (data2.A == A), 'ta'].cumsum()
        for H in data2.H.unique():
            data2.loc[(data2.S == S) & (data2.H == H), 'th'] = data2.loc[(data2.S == S) & (data2.H == H), 'th'].cumsum()

In [13]:
# задание кластеров в зависимости от p3.1 и p4.1
def claster_42(data_42: pd.DataFrame):
    # выборка из полного датасета таких строк, где TA == 42 или 38 в зависимости от S
    data_42 = pd.concat([data_42[data_42.TA == 42], data_42[(data_42.S > 2) & (data_42.TA == 38)]])
    data_42['KH'], data_42['KA'] = 0, 0
    
    data_42.loc[(data_42['p3.1'] >= 80) & (data_42['p3.1'] <= 114), 'KH'] = 1
    data_42.loc[(data_42['p3.1'] >= 65) & (data_42['p3.1'] <= 79), 'KH'] = 2
    data_42.loc[(data_42['p3.1'] >= 55) & (data_42['p3.1'] <= 64), 'KH'] = 3
    data_42.loc[(data_42['p3.1'] >= 41) & (data_42['p3.1'] <= 54), 'KH'] = 4
    data_42.loc[(data_42['p3.1'] >= 0) & (data_42['p3.1'] <= 40), 'KH'] = 5

    data_42.loc[(data_42['p4.1'] >= 80) & (data_42['p4.1'] <= 114), 'KA'] = 1
    data_42.loc[(data_42['p4.1'] >= 65) & (data_42['p4.1'] <= 79), 'KA'] = 2
    data_42.loc[(data_42['p4.1'] >= 55) & (data_42['p4.1'] <= 64), 'KA'] = 3
    data_42.loc[(data_42['p4.1'] >= 41) & (data_42['p4.1'] <= 54), 'KA'] = 4
    data_42.loc[(data_42['p4.1'] >= 0) & (data_42['p4.1'] <= 40), 'KA'] = 5
    
    return data_42

In [14]:
def claster_19A(data: pd.DataFrame):
    data = pd.concat([data[data.ta == 21], data[(data.S > 2) & (data.ta == 19)]])
    data['Ka'] = 0
    data.loc[(data['p2.1'] >= 40) & (data['p2.1'] <= 57), 'Ka'] = 1
    data.loc[(data['p2.1'] >= 35) & (data['p2.1'] <= 39), 'Ka'] = 2
    data.loc[(data['p2.1'] >= 25) & (data['p2.1'] <= 34), 'Ka'] = 3
    data.loc[(data['p2.1'] >= 16) & (data['p2.1'] <= 24), 'Ka'] = 4
    data.loc[(data['p2.1'] >= 0) & (data['p2.1'] <= 15), 'Ka'] = 5
    return data

In [15]:
def claster_19H(data: pd.DataFrame):
    data = pd.concat([data[data.th == 21], data[(data.S > 2) & (data.th == 19)]])
    
    data.loc[(data['p1.1'] >= 40) & (data['p1.1'] <= 57), 'Kh'] = 1
    data.loc[(data['p1.1'] >= 35) & (data['p1.1'] <= 39), 'Kh'] = 2
    data.loc[(data['p1.1'] >= 30) & (data['p1.1'] <= 34), 'Kh'] = 3
    data.loc[(data['p1.1'] >= 21) & (data['p1.1'] <= 29), 'Kh'] = 4
    data.loc[(data['p1.1'] >= 0) & (data['p1.1'] <= 20), 'Kh'] = 5
    return data

In [20]:
def create_KAH(df):
    dh, da = claster_42(df)[['S', 'H', 'KH', 'id']], claster_42(df)[['S', 'A', 'KA', 'id']]
    da.rename(columns={'A': 'H', 'KA': 'KH'}, inplace=True)
    dh = pd.concat([dh, da])
    dh.sort_values('id', inplace=True)
    dh.reset_index(drop=True, inplace=True)
    dh.rename(columns={'KH': 'K', 'H': 'HA'}, inplace=True)

    dh = dh[['S', 'HA', 'K']]
    dh['KK'] = 0

    for HA in dh.HA.unique():
        dh.loc[dh.HA == HA, 'KK'] = dh.loc[dh.HA == HA, 'K'].shift()

    dh.loc[(dh.S == 1), 'KK'] = dh.loc[(dh.S == 1), 'KK'].fillna(0)
    dh.loc[(dh.S == 1), 'KK'] = dh.loc[(dh.S == 1), 'K'] + dh.loc[(dh.S == 1), 'KK']
    dh.fillna(5, inplace=True)

    d = dh[['S', 'HA', 'KK']]
    d.rename(columns={'HA':'H'}, inplace=True)
    d.fillna(5, inplace=True)

    da = dh[['S', 'HA', 'KK']]
    da.rename(columns={'HA':'A'}, inplace=True)
    da.fillna(5, inplace=True)

    df = df.merge(d, left_on=['S', 'H'], right_on=['S', 'H'])
    df = df.merge(da, left_on=['S', 'A'], right_on=['S', 'A'])
    df.rename(columns={'KK_x' : 'KH', 'KK_y': 'KA'}, inplace=True)
    return df

In [38]:
def create_Kaa_and_Khh(df: pd.DataFrame):
    da = claster_19A(df)[['S', 'A', 'Ka']]
    da['Kaa'] = 0

    for A in da.A.unique():
        da.loc[da.A == A, 'Kaa'] = da.loc[da.A == A, 'Ka'].shift()

    da.loc[(da.S == 1), 'Kaa'] = da.loc[(da.S == 1), 'Kaa'].fillna(0)
    da.loc[(da.S == 1), 'Kaa'] = da.loc[(da.S == 1), 'Ka'] + da.loc[(da.S == 1), 'Kaa']

    da.fillna(5, inplace=True)
    return da

In [39]:
create_Kaa_and_Khh(df)

AttributeError: 'DataFrame' object has no attribute 'ta'

In [40]:
df

,S,H,A,TH,TA,gh,ga,allg,ph,pa,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,y1,y2,p1,p2,p5,p6,id,p3,p4,p7,p8,p1.1,p2.1,p3.1,p4.1,KH,KA,last_h5,last_a5,last_5KH,last_5KA
0,1,0,15,1,1,0,3,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,0.0,3.000000,3.00,3.0
1,1,1,15,7,7,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6.0,5.0,6.0,5.0,68,11.0,12.0,11.0,12.0,7.0,6.0,12.0,13.0,3.0,3.0,0.0,1.500000,1.50,0.0
2,1,14,15,41,41,1,2,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,35.0,18.0,35.0,18.0,449,48.0,52.0,48.0,52.0,35.0,21.0,48.0,55.0,4.0,3.0,1.0,1.666667,2.00,3.0
3,1,25,15,30,31,1,0,1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,28.0,14.0,28.0,14.0,316,44.0,38.0,44.0,38.0,31.0,14.0,47.0,38.0,3.0,3.0,1.0,1.250000,1.75,0.5
4,1,26,15,5,5,1,1,2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.0,4.0,1.0,4.0,45,1.0,8.0,1.0,8.0,2.0,5.0,2.0,9.0,4.0,3.0,1.0,1.200000,1.80,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,28,24,19,10,10,1,2,3,0,3,16.0,10.0,3.0,5.0,6.0,5.0,9.0,21.0,0.0,2.0,0.0,0.0,0,1,6.0,1.0,403.0,200.0,10521,18.0,4.0,687.0,616.0,6.0,4.0,18.0,7.0,3.0,5.0,1.6,0.800000,2.40,3.0
10800,28,1,19,29,31,3,1,4,3,0,14.0,15.0,4.0,7.0,4.0,4.0,14.0,20.0,1.0,2.0,0.0,0.0,1,1,17.0,16.0,700.0,215.0,10721,41.0,26.0,1225.0,638.0,20.0,16.0,44.0,26.0,5.0,5.0,0.6,1.000000,2.60,4.2
10801,28,49,19,4,4,1,0,1,3,0,14.0,10.0,5.0,2.0,6.0,2.0,15.0,8.0,1.0,1.0,0.0,0.0,0,0,0.0,0.0,152.0,199.0,10460,3.0,0.0,255.0,612.0,3.0,0.0,6.0,0.0,3.0,5.0,0.8,1.000000,3.00,3.0
10802,28,11,19,24,24,1,1,2,1,1,8.0,8.0,3.0,3.0,5.0,5.0,7.0,13.0,1.0,1.0,0.0,0.0,0,0,14.0,11.0,169.0,210.0,10660,26.0,18.0,277.0,630.0,15.0,12.0,27.0,19.0,4.0,5.0,0.6,0.800000,2.40,2.0


In [32]:
%%time
def last_h5(DF: pd.DataFrame, window_size: int):
    DF['last_h5'] = 0
    for S in DF.S.unique():
        for H in DF[DF.S == S].H.unique():
            numbers_series = pd.Series(DF[(DF.S == S) & (DF.H == H)].gh)
            windows = numbers_series.rolling(window_size)
            moving_averages = windows.mean()
            DF.loc[(DF.S == S) & (DF.H == H), 'last_h5'] = moving_averages.shift()
            a = []
            for i in range (1, window_size + 1):
                a.append((DF.loc[(DF.S == S) & (DF.H == H) & (DF.last_h5.isna())].gh).to_numpy()[:i].mean())
            DF.loc[(DF.last_h5.isna())&(DF.S == S) & (DF.H == H), 'last_h5'] = a
    # print(a)
    return DF

def last_a5(DF: pd.DataFrame, window_size: int):
    DF['last_a5'] = 0
    for S in DF.S.unique():
        for A in DF[DF.S == S].A.unique():
            numbers_series = pd.Series(DF[(DF.S == S) & (DF.A == A)].ga)
            windows = numbers_series.rolling(window_size)
            moving_averages = windows.mean()
            DF.loc[(DF.S == S) & (DF.A == A), 'last_a5'] = moving_averages.shift()
            a = []
            for i in range (1, window_size + 1):
                a.append((DF.loc[(DF.S == S) & (DF.A == A) & (DF.last_a5.isna())].ga).to_numpy()[:i].mean())
            DF.loc[(DF.last_a5.isna())&(DF.S == S) & (DF.A == A), 'last_a5'] = a
    # print(a)
    return DF

Wall time: 0 ns


In [34]:
def last_5KH(df: pd.DataFrame, window_size: int):
    df['last_5KH'] = 0
    for ind in df.index:
        KA = df.iloc[ind].KA
        numbers_series = pd.Series(df[df.KA == KA].allg)
        windows = numbers_series.rolling(window_size)
        moving_averages = windows.mean()
        df.loc[df.KA == KA, 'last_5KH'] = moving_averages.shift()
        a = []
        for i in range (1, window_size + 1):
            a.append((df.loc[(df.KA == KA) & (df.last_5KH.isna())].allg).to_numpy()[:i].mean())
        df.loc[(df.last_5KH.isna())&(df.KA == KA), 'last_5KH'] = a
    # print(a)
    return df

def last_5KA(df: pd.DataFrame, window_size: int):
    df['last_5KA'] = 0
    for ind in df.index:
        KH = df.iloc[ind].KH
        numbers_series = pd.Series(df[df.KH == KH].allg)
        windows = numbers_series.rolling(window_size)
        moving_averages = windows.mean()
        df.loc[df.KH == KH, 'last_5KA'] = moving_averages.shift()
        a = []
        for i in range (1, window_size + 1):
            a.append((df.loc[(df.KH == KH) & (df.last_5KA.isna())].allg).to_numpy()[:i].mean())
        df.loc[(df.last_5KA.isna())&(df.KH == KH), 'last_5KA'] = a
    # print(a)
    return df

In [36]:
%%time
df = cumsum_os_ha(df)
df = cumsum_o_ha(df)
df = cumsum_o(df)
df = create_p_1234(df)
df = function_T(df)
df = create_KAH(df)
df = last_h5(df, 5)
df = last_a5(df, 5)
df = last_5KH(df, 5)
df = last_5KA(df, 5)

Wall time: 2min 10s


In [43]:
df.to_csv('features.csv', index=False)

In [44]:
df

,S,H,A,TH,TA,gh,ga,allg,ph,pa,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,y1,y2,p1,p2,p5,p6,id,p3,p4,p7,p8,p1.1,p2.1,p3.1,p4.1,KH,KA,last_h5,last_a5,last_5KH,last_5KA
0,1,0,15,1,1,0,3,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,0.0,3.000000,3.00,3.0
1,1,1,15,7,7,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6.0,5.0,6.0,5.0,68,11.0,12.0,11.0,12.0,7.0,6.0,12.0,13.0,3.0,3.0,0.0,1.500000,1.50,0.0
2,1,14,15,41,41,1,2,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,35.0,18.0,35.0,18.0,449,48.0,52.0,48.0,52.0,35.0,21.0,48.0,55.0,4.0,3.0,1.0,1.666667,2.00,3.0
3,1,25,15,30,31,1,0,1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,28.0,14.0,28.0,14.0,316,44.0,38.0,44.0,38.0,31.0,14.0,47.0,38.0,3.0,3.0,1.0,1.250000,1.75,0.5
4,1,26,15,5,5,1,1,2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.0,4.0,1.0,4.0,45,1.0,8.0,1.0,8.0,2.0,5.0,2.0,9.0,4.0,3.0,1.0,1.200000,1.80,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,28,24,19,10,10,1,2,3,0,3,16.0,10.0,3.0,5.0,6.0,5.0,9.0,21.0,0.0,2.0,0.0,0.0,0,1,6.0,1.0,403.0,200.0,10521,18.0,4.0,687.0,616.0,6.0,4.0,18.0,7.0,3.0,5.0,1.6,0.800000,2.40,3.0
10800,28,1,19,29,31,3,1,4,3,0,14.0,15.0,4.0,7.0,4.0,4.0,14.0,20.0,1.0,2.0,0.0,0.0,1,1,17.0,16.0,700.0,215.0,10721,41.0,26.0,1225.0,638.0,20.0,16.0,44.0,26.0,5.0,5.0,0.6,1.000000,2.60,4.2
10801,28,49,19,4,4,1,0,1,3,0,14.0,10.0,5.0,2.0,6.0,2.0,15.0,8.0,1.0,1.0,0.0,0.0,0,0,0.0,0.0,152.0,199.0,10460,3.0,0.0,255.0,612.0,3.0,0.0,6.0,0.0,3.0,5.0,0.8,1.000000,3.00,3.0
10802,28,11,19,24,24,1,1,2,1,1,8.0,8.0,3.0,3.0,5.0,5.0,7.0,13.0,1.0,1.0,0.0,0.0,0,0,14.0,11.0,169.0,210.0,10660,26.0,18.0,277.0,630.0,15.0,12.0,27.0,19.0,4.0,5.0,0.6,0.800000,2.40,2.0
